In [4]:
from os import fsencode, listdir, fsdecode
from PIL import Image, ImageDraw

directory = fsencode('input')
for file in listdir(directory):
    filename = fsdecode(file)
    if filename.endswith('.jpg'):
        # print('Filename {}'.format(filename))
        img = Image.open('input/' + filename)
        w, h = img.size
        # img.show()
        # With all of this set we can crop about 10% from bottom to delete rulers
        crop_mult_y = 0.9
        # There are 2 base types of scans, detect which are we dealing with
        # One type should have characteristic blue (2, 143, 199) at about 4% width
        is_transparent_ruler_type = True
        for i in range(0, 500):
            sample = img.getpixel((0.042 * w, i))
            if 0 <= sample[0] <= 5 and 130 <= sample[1] <= 150 and 170 <= sample[2] <= 210:
                is_transparent_ruler_type = False
                break
        # print(is_transparent_ruler_type)
        if is_transparent_ruler_type:
            crop_mult_x = 0.07
        else:
            crop_mult_x = 0.11
        img = img.crop((w * crop_mult_x, 0, w, h * crop_mult_y))
        w, h = img.size
        # img.show()
        # Sample 10 pixels from where there should always be solid background
        samples = 0
        for i in range(1000, 1030):
            samples += sum(img.getpixel((i, 50))) / 3
        sample_average = samples / 30
        # print('Sample average is {}'.format(sample_average))
        center_x = int(w / 2)
        center_y = int(h / 2)
        
        upper_edge_y = None
        for i in range(0, h):
            average_pixel_rgb = sum(img.getpixel((center_x, i))) / 3
            # print (average_pixel_rgb)
            if not (sample_average * 0.97 < average_pixel_rgb < sample_average * 1.03):
                # We have likely hit the upper edge of the real image
                # draw = ImageDraw.Draw(img)
                # draw.line((0, i, w, i), fill=128)
                # img.show()
                upper_edge_y = i
                break
    
        lower_edge_y = None
        for i in range(h - 1, 0, -1):
            average_pixel_rgb = sum(img.getpixel((center_x, i))) / 3
            # print (average_pixel_rgb)
            if not (sample_average * 0.95 < average_pixel_rgb < sample_average * 1.05):
                # We have likely hit the lower edge of the real image
                # draw = ImageDraw.Draw(img)
                # draw.line((0, i, w, i), fill=128)
                # img.show()
                lower_edge_y = i
                break
    
        right_edge_x = None
        for i in range(w - 1, 0, -1):
            average_pixel_rgb = sum(img.getpixel((i, center_y))) / 3
            # print (average_pixel_rgb)
            if not (sample_average * 0.95 < average_pixel_rgb < sample_average * 1.05):
                # We have likely hit the right edge of the real image
                # draw = ImageDraw.Draw(img)
                # draw.line((i, 0, i, h), fill=128)
                # img.show()
                right_edge_x = i
                break
                
        left_edge_x = None
        for i in range(0, w):
            average_pixel_rgb = sum(img.getpixel((i, center_y))) / 3
            # print (average_pixel_rgb)
            if not (sample_average * 0.95 < average_pixel_rgb < sample_average * 1.05):
                # We have likely hit the left edge of the real image
                # draw = ImageDraw.Draw(img)
                # draw.line((i, 0, i, h), fill=128)
                # img.show()
                left_edge_x = i
                break
    
        # img.show()
        img = img.crop((left_edge_x, upper_edge_y, right_edge_x, lower_edge_y))
        with open('output/' + filename, 'w') as f:
            img.save(f)

